In [1]:
import random
import cv2
import math
import imageio
from sklearn.metrics.cluster import entropy

MAX_NUMBER_OF_FRAMES = 100

TOTAL_KEY_FRAMES = 10

STOPPING_ITERATION = 40

NUMBER_OF_NP_CANDIDATES = 10

# Location to read images from.
# NOTE: "_" is used to follow a naming convention
# eg. _20.jpg, _21.jpg etc...
READ_LOCATION = "C:/Users/Brian/Desktop/Desktop/Thesis/IMDb_Scrape/Movie_Trailers/Deadpool_2/frames/_"

# Location to write GIF images to.
GIF_WRITE_LOCATION = "C:/Users/Brian/Desktop/Desktop/Thesis/IMDb_Scrape/Movie_Trailers/Deadpool_2/key_frames/avg_entropy.GIF"

# Population matrix.
NP = []

# Mutation vector.
MV = []

# Trail vector.
TV = []

# Scale factor.
F = 0.9

# Cr probability value.
Cr = 0.6

# Calculate Average Entropy Difference for a chromosome.
def getEntropy( KF ):
    entropy_sum = 0
    for i in range(1, TOTAL_KEY_FRAMES - 1):
        while True:
            try:
                im1 = cv2.imread(READ_LOCATION + str(KF[i]) + ".jpg",0)
                im2 = cv2.imread(READ_LOCATION + str(KF[i+1]) + ".jpg",0)
                entropy_sum += abs(entropy(im1) - entropy(im2))
            except:
                print(i, KF, KF[i], KF[i+1])
                continue
            break
    return entropy_sum/(TOTAL_KEY_FRAMES - 1)

# INITIALISATION : Generates population NP of 10 parent vectors (and Average Entropy Differences).
def initialize_NP():
    for i in range(NUMBER_OF_NP_CANDIDATES):
        NP.append(sorted(random.sample(range(1, MAX_NUMBER_OF_FRAMES+1), TOTAL_KEY_FRAMES)))
        NP[-1].append(getEntropy(NP[-1]))
        print(NP[-1])

# MUTATION
def mutation(parent):
    R = random.sample(NP,2)
    global MV
    MV[:] = []
    MV_value = 0
    print(NP[parent])
    for i in range(TOTAL_KEY_FRAMES):
        MV_value = int(NP[parent][i] + F*(R[0][i] - R[1][i]))
        if(MV_value < 1):
            MV.append(1)
        elif(MV_value > MAX_NUMBER_OF_FRAMES):
            MV.append(MAX_NUMBER_OF_FRAMES)
        else:
            MV.append(MV_value)
    MV.sort()
    MV.append(getEntropy(MV))

# CROSSOVER (uniform crossover with Cr = 0.6).
def crossover(parent, mutant):
    print("mutant: ", mutant)
    print("parent: ", parent)
    for j in range(TOTAL_KEY_FRAMES) :
        if(random.uniform(0,1) < Cr) :
            TV.append(mutant[j])
        else:
            TV.append(parent[j])
    TV.sort()
    TV.append(getEntropy(TV))
    print("TV    : ", TV)

# SELECTION : Selects offspring / parent based on higher Entropy diff. value.
def selection(parent, trail_vector):
    if(trail_vector[-1] > parent[-1]):
        parent[:] = trail_vector
        print("yes", parent)
    else:
        print("no")

# bestParent returns the parent with then maximum Entropy diff. value.
def bestParent(population):
    Max_Entropy_value = population[0][-1]
    Best_Parent_Index = population[0]
    for parent in population:
        if (parent[-1] > Max_Entropy_value):
            Max_Entropy_value = parent[-1]
            Best_Parent_Index = parent
    return Best_Parent_Index

initialize_NP()
for GENERATION in range(STOPPING_ITERATION):
    for i in range(NUMBER_OF_NP_CANDIDATES):
        print("---------------------", "PARENT:", i+1 , "GENERATION:", GENERATION+1, "---------------------")
        mutation(i)
        crossover(NP[i], MV)
        selection(NP[i], TV)
        print(NP[i])
        TV[:] = []
        print("")
    print("")
best_parent = bestParent(NP)
print("best solution is: ", best_parent)
images_for_gif = []
for frame_number in best_parent[:-1]:
        images_for_gif.append(imageio.imread(READ_LOCATION + str(frame_number) + '.jpg'))
imageio.mimsave(GIF_WRITE_LOCATION, images_for_gif)

[23, 31, 32, 46, 51, 73, 78, 83, 88, 98, 0.24632093531276736]
[26, 37, 42, 49, 50, 56, 61, 63, 87, 92, 0.15863236441216547]
[1, 5, 7, 20, 31, 43, 65, 88, 89, 93, 0.2503728385819015]
[3, 4, 9, 19, 25, 26, 38, 41, 73, 78, 0.0910799299760595]
[7, 18, 32, 42, 56, 62, 75, 82, 92, 95, 0.24249200174105578]
[21, 26, 37, 38, 44, 64, 76, 82, 84, 92, 0.29080792378087605]
[5, 23, 41, 42, 49, 50, 52, 58, 68, 89, 0.22865037433183572]
[23, 24, 36, 37, 56, 57, 71, 73, 74, 99, 0.24393674051892006]
[2, 19, 23, 30, 41, 50, 53, 58, 74, 86, 0.22693415017818117]
[11, 21, 31, 37, 38, 45, 60, 64, 83, 92, 0.2216846187566474]
--------------------- PARENT: 1 GENERATION: 1 ---------------------
[23, 31, 32, 46, 51, 73, 78, 83, 88, 98, 0.24632093531276736]
mutant:  [44, 47, 49, 59, 63, 78, 85, 87, 99, 100, 0.20336008402726868]
parent:  [23, 31, 32, 46, 51, 73, 78, 83, 88, 98, 0.24632093531276736]
TV    :  [32, 44, 47, 59, 63, 73, 83, 85, 88, 98, 0.3396328077637292]
yes [32, 44, 47, 59, 63, 73, 83, 85, 88, 98, 0.33

mutant:  [1, 1, 1, 15, 17, 18, 24, 27, 64, 80, 0.23441425456212034]
parent:  [2, 19, 23, 30, 41, 50, 53, 58, 74, 86, 0.22693415017818117]
TV    :  [1, 2, 19, 30, 41, 50, 53, 58, 64, 80, 0.21861973452834318]
no
[2, 19, 23, 30, 41, 50, 53, 58, 74, 86, 0.22693415017818117]

--------------------- PARENT: 10 GENERATION: 2 ---------------------
[11, 21, 31, 37, 38, 45, 60, 64, 83, 92, 0.2216846187566474]
mutant:  [5, 13, 15, 21, 33, 40, 55, 58, 66, 96, 0.28472635692979326]
parent:  [11, 21, 31, 37, 38, 45, 60, 64, 83, 92, 0.2216846187566474]
TV    :  [11, 13, 21, 31, 38, 40, 55, 64, 83, 92, 0.21974626429393274]
no
[11, 21, 31, 37, 38, 45, 60, 64, 83, 92, 0.2216846187566474]


--------------------- PARENT: 1 GENERATION: 3 ---------------------
[32, 44, 47, 59, 63, 73, 83, 85, 88, 98, 0.3396328077637292]
mutant:  [14, 34, 37, 51, 60, 60, 62, 63, 79, 92, 0.284428275139182]
parent:  [32, 44, 47, 59, 63, 73, 83, 85, 88, 98, 0.3396328077637292]
TV    :  [14, 44, 47, 59, 60, 63, 63, 79, 83, 98, 0.3

TV    :  [23, 24, 36, 37, 52, 56, 65, 67, 78, 99, 0.2803942124172465]
yes [23, 24, 36, 37, 52, 56, 65, 67, 78, 99, 0.2803942124172465]
[23, 24, 36, 37, 52, 56, 65, 67, 78, 99, 0.2803942124172465]

--------------------- PARENT: 9 GENERATION: 4 ---------------------
[4, 22, 30, 39, 48, 50, 52, 58, 68, 86, 0.2301123055989571]
mutant:  [15, 45, 46, 49, 50, 59, 63, 63, 63, 94, 0.17474143470142436]
parent:  [4, 22, 30, 39, 48, 50, 52, 58, 68, 86, 0.2301123055989571]
TV    :  [15, 30, 39, 45, 50, 52, 59, 63, 68, 94, 0.2959923502002821]
yes [15, 30, 39, 45, 50, 52, 59, 63, 68, 94, 0.2959923502002821]
[15, 30, 39, 45, 50, 52, 59, 63, 68, 94, 0.2959923502002821]

--------------------- PARENT: 10 GENERATION: 4 ---------------------
[1, 1, 1, 11, 21, 45, 60, 64, 80, 92, 0.2480707566724658]
mutant:  [14, 37, 48, 50, 64, 85, 98, 100, 100, 100, 0.25650883447543954]
parent:  [1, 1, 1, 11, 21, 45, 60, 64, 80, 92, 0.2480707566724658]
TV    :  [1, 11, 37, 48, 64, 64, 85, 92, 98, 100, 0.2654694177695069]


mutant:  [6, 10, 44, 58, 73, 83, 94, 94, 100, 100, 0.2685578839305252]
parent:  [23, 24, 36, 37, 52, 56, 65, 67, 78, 99, 0.2803942124172465]
TV    :  [10, 23, 44, 56, 58, 73, 78, 94, 94, 99, 0.2560895384666251]
no
[23, 24, 36, 37, 52, 56, 65, 67, 78, 99, 0.2803942124172465]

--------------------- PARENT: 9 GENERATION: 6 ---------------------
[15, 30, 39, 45, 50, 52, 59, 63, 68, 94, 0.2959923502002821]
mutant:  [15, 39, 81, 91, 98, 100, 100, 100, 100, 100, 0.13659078279567696]
parent:  [15, 30, 39, 45, 50, 52, 59, 63, 68, 94, 0.2959923502002821]
TV    :  [15, 30, 39, 50, 52, 68, 91, 94, 100, 100, 0.22115184675157204]
no
[15, 30, 39, 45, 50, 52, 59, 63, 68, 94, 0.2959923502002821]

--------------------- PARENT: 10 GENERATION: 6 ---------------------
[1, 11, 48, 69, 74, 82, 92, 93, 98, 100, 0.436711997222461]
mutant:  [1, 2, 11, 20, 35, 51, 71, 88, 93, 93, 0.1290570915935946]
parent:  [1, 11, 48, 69, 74, 82, 92, 93, 98, 100, 0.436711997222461]
TV    :  [1, 11, 20, 35, 48, 71, 82, 88, 93, 

mutant:  [46, 51, 51, 52, 56, 56, 58, 64, 67, 100, 0.2205118892930453]
parent:  [23, 24, 36, 37, 52, 56, 65, 67, 78, 99, 0.2803942124172465]
TV    :  [23, 36, 51, 52, 52, 56, 58, 64, 78, 99, 0.2569479814148194]
no
[23, 24, 36, 37, 52, 56, 65, 67, 78, 99, 0.2803942124172465]

--------------------- PARENT: 9 GENERATION: 8 ---------------------
[34, 39, 44, 45, 50, 59, 64, 65, 68, 94, 0.38127180387106185]
mutant:  [46, 50, 52, 53, 56, 57, 58, 65, 66, 100, 0.36642226183801774]
parent:  [34, 39, 44, 45, 50, 59, 64, 65, 68, 94, 0.38127180387106185]
TV    :  [39, 44, 45, 46, 50, 57, 58, 65, 66, 100, 0.3625504016701384]
no
[34, 39, 44, 45, 50, 59, 64, 65, 68, 94, 0.38127180387106185]

--------------------- PARENT: 10 GENERATION: 8 ---------------------
[1, 11, 48, 69, 74, 82, 92, 93, 98, 100, 0.436711997222461]
mutant:  [1, 1, 19, 49, 74, 91, 96, 100, 100, 100, 0.15978058367693176]
parent:  [1, 11, 48, 69, 74, 82, 92, 93, 98, 100, 0.436711997222461]
TV    :  [1, 1, 48, 49, 74, 91, 96, 100, 100

...

mutant:  [1, 1, 1, 15, 19, 45, 65, 80, 84, 85, 0.27970491266034714]
parent:  [1, 1, 8, 43, 63, 65, 74, 93, 98, 100, 0.4572909377898822]
TV    :  [1, 1, 8, 43, 45, 63, 74, 80, 84, 100, 0.3722105277168635]
no
[1, 1, 8, 43, 63, 65, 74, 93, 98, 100, 0.4572909377898822]


--------------------- PARENT: 1 GENERATION: 31 ---------------------
[7, 34, 44, 57, 63, 68, 69, 79, 88, 98, 0.6217112663294854]
mutant:  [1, 1, 7, 30, 45, 50, 53, 63, 71, 84, 0.27207482917099896]
parent:  [7, 34, 44, 57, 63, 68, 69, 79, 88, 98, 0.6217112663294854]
TV    :  [7, 34, 44, 50, 57, 63, 69, 71, 79, 98, 0.4116710843065134]
no
[7, 34, 44, 57, 63, 68, 69, 79, 88, 98, 0.6217112663294854]

--------------------- PARENT: 2 GENERATION: 31 ---------------------
[24, 25, 39, 41, 42, 49, 65, 73, 85, 98, 0.43615761247005125]
mutant:  [11, 17, 24, 25, 42, 44, 60, 73, 87, 98, 0.26134845240439275]
parent:  [24, 25, 39, 41, 42, 49, 65, 73, 85, 98, 0.43615761247005125]
TV    :  [24, 24, 25, 41, 42, 44, 65, 73, 87, 98, 0.43297986

TV    :  [1, 1, 8, 36, 43, 56, 65, 73, 87, 98, 0.4580158157373854]
yes [1, 1, 8, 36, 43, 56, 65, 73, 87, 98, 0.4580158157373854]
[1, 1, 8, 36, 43, 56, 65, 73, 87, 98, 0.4580158157373854]


--------------------- PARENT: 1 GENERATION: 33 ---------------------
[7, 34, 44, 57, 63, 68, 69, 79, 88, 98, 0.6217112663294854]
mutant:  [7, 69, 82, 83, 86, 86, 95, 95, 97, 99, 0.10763999953090991]
parent:  [7, 34, 44, 57, 63, 68, 69, 79, 88, 98, 0.6217112663294854]
TV    :  [7, 34, 57, 63, 82, 86, 88, 95, 95, 98, 0.18869902438280958]
no
[7, 34, 44, 57, 63, 68, 69, 79, 88, 98, 0.6217112663294854]

--------------------- PARENT: 2 GENERATION: 33 ---------------------
[24, 25, 39, 41, 42, 49, 65, 73, 85, 98, 0.43615761247005125]
mutant:  [1, 1, 13, 24, 25, 26, 53, 57, 72, 83, 0.19135290121627355]
parent:  [24, 25, 39, 41, 42, 49, 65, 73, 85, 98, 0.43615761247005125]
TV    :  [1, 24, 25, 25, 26, 39, 57, 65, 72, 83, 0.39656622465356783]
no
[24, 25, 39, 41, 42, 49, 65, 73, 85, 98, 0.43615761247005125]

--

mutant:  [11, 34, 44, 63, 68, 69, 77, 85, 88, 98, 0.5064709059681345]
parent:  [7, 34, 44, 57, 63, 68, 69, 79, 88, 98, 0.6217112663294854]
TV    :  [7, 34, 44, 57, 68, 69, 69, 79, 88, 98, 0.41332567333794906]
no
[7, 34, 44, 57, 63, 68, 69, 79, 88, 98, 0.6217112663294854]

--------------------- PARENT: 2 GENERATION: 35 ---------------------
[24, 25, 39, 41, 42, 49, 65, 73, 85, 98, 0.43615761247005125]
mutant:  [17, 28, 32, 34, 41, 43, 61, 73, 82, 98, 0.28262029894453533]
parent:  [24, 25, 39, 41, 42, 49, 65, 73, 85, 98, 0.43615761247005125]
TV    :  [24, 25, 32, 34, 42, 49, 61, 73, 82, 98, 0.29167040496892277]
no
[24, 25, 39, 41, 42, 49, 65, 73, 85, 98, 0.43615761247005125]

--------------------- PARENT: 3 GENERATION: 35 ---------------------
[1, 1, 33, 33, 40, 63, 72, 96, 100, 100, 0.3663971954705101]
mutant:  [6, 30, 51, 58, 65, 73, 75, 100, 100, 100, 0.3310422138593462]
parent:  [1, 1, 33, 33, 40, 63, 72, 96, 100, 100, 0.3663971954705101]
TV    :  [1, 30, 33, 58, 65, 73, 75, 96, 100,

TV    :  [7, 44, 62, 63, 69, 73, 82, 88, 92, 100, 0.4043899225754546]
no
[7, 34, 44, 57, 63, 68, 69, 79, 88, 98, 0.6217112663294854]

--------------------- PARENT: 2 GENERATION: 37 ---------------------
[24, 25, 39, 41, 42, 49, 65, 73, 85, 98, 0.43615761247005125]
mutant:  [8, 33, 43, 55, 60, 66, 68, 78, 87, 98, 0.47351217018277186]
parent:  [24, 25, 39, 41, 42, 49, 65, 73, 85, 98, 0.43615761247005125]
TV    :  [8, 33, 41, 43, 49, 60, 65, 78, 87, 98, 0.2986202837983262]
no
[24, 25, 39, 41, 42, 49, 65, 73, 85, 98, 0.43615761247005125]

--------------------- PARENT: 3 GENERATION: 37 ---------------------
[1, 30, 33, 58, 65, 73, 75, 96, 100, 100, 0.4053536361715416]
mutant:  [1, 30, 33, 36, 44, 49, 66, 87, 89, 91, 0.2356614359554484]
parent:  [1, 30, 33, 58, 65, 73, 75, 96, 100, 100, 0.4053536361715416]
TV    :  [1, 30, 33, 36, 65, 66, 73, 91, 96, 100, 0.4094007972308195]
yes [1, 30, 33, 36, 65, 66, 73, 91, 96, 100, 0.4094007972308195]
[1, 30, 33, 36, 65, 66, 73, 91, 96, 100, 0.4094007972

mutant:  [25, 28, 39, 47, 47, 58, 65, 79, 85, 98, 0.43369612144460323]
parent:  [24, 25, 39, 41, 42, 49, 65, 73, 85, 98, 0.43615761247005125]
TV    :  [24, 28, 39, 47, 47, 58, 65, 73, 85, 98, 0.4276699741890535]
no
[24, 25, 39, 41, 42, 49, 65, 73, 85, 98, 0.43615761247005125]

--------------------- PARENT: 3 GENERATION: 39 ---------------------
[1, 30, 33, 36, 65, 66, 73, 91, 96, 100, 0.4094007972308195]
mutant:  [1, 8, 24, 58, 83, 83, 87, 100, 100, 100, 0.12097719817584668]
parent:  [1, 30, 33, 36, 65, 66, 73, 91, 96, 100, 0.4094007972308195]
TV    :  [1, 24, 30, 58, 65, 66, 73, 91, 100, 100, 0.412976058162682]
yes [1, 24, 30, 58, 65, 66, 73, 91, 100, 100, 0.412976058162682]
[1, 24, 30, 58, 65, 66, 73, 91, 100, 100, 0.412976058162682]

--------------------- PARENT: 4 GENERATION: 39 ---------------------
[1, 1, 1, 36, 38, 69, 74, 86, 92, 98, 0.4404389126906033]
mutant:  [1, 1, 1, 22, 22, 40, 56, 66, 81, 96, 0.26705647902246726]
parent:  [1, 1, 1, 36, 38, 69, 74, 86, 92, 98, 0.440438912